In [16]:
%pip install requests 

Note: you may need to restart the kernel to use updated packages.


In [17]:
import requests
from bs4 import BeautifulSoup

def content_scraper(soup, links, identifier):
    link_and_data = {}

    for link in links:
        response = requests.get(links[0])

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            match identifier:
                case 'jena':
                    title = soup.find_all('title')
                    content = soup.find_all('div', class_='flex-layout')

                    text_content = title[0].get_text(strip=True) + ': ' + content[0].get_text(strip=True)
                case 'siegburg':
                    divs = soup.find_all('div', class_='flex-layout')
                    content = divs[1]

                    text_content = content.get_text(strip=True)
                case _:
                    text_content = "Wrong identifier"

            link_and_data[link] = text_content

        else:
            message = "Failed to retrieve the webpage. Status code:", response.status_code
            link_and_data[link] = message
    
    return link_and_data

In [18]:
def siegburg_data(soup):
    links = []

    a_tags = soup.find_all('a')

    for tag in a_tags:
        href = tag.get('href')
        
        if href and '/proposals/' in href and not 'new' in href:
            link = 'https://mitmachen.siegburg.de' + href
    
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'siegburg')

    return link_and_content


def jena_data(soup):
    links = []

    a_tags = soup.find_all('a',  class_='resource-item--title')

    for tag in a_tags:
        href = tag.get('href')
        
        if href:
            link = 'https://mitmachen.jena.de' + href
            
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'jena')

    return link_and_content

In [ ]:
urls = [
        'https://mitmachen.siegburg.de/angebotslandkarte',
        'https://mitmachen.jena.de/projekts'
    ]


def def_42(urls):
    urls_and_data = {}

    for url in urls:
        response = requests.get(url)
        if response.status_code != 200:
            print("Failed to retrieve the webpage. Status code:", response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        link_and_content = scraper(soup, url)
        urls_and_data[url] = link_and_content

    return urls_and_data


def scraper(soup, url):
    if 'siegburg' in url:
        return siegburg_data(soup)
    elif 'jena' in url:
        return jena_data(soup)
    
    return "No scraper defined for this URL"


link_and_data = def_42(urls)

In [ ]:
link_and_data[urls[1]]

KeyError: 'https://mitmachen.jena.de/projekts'

In [ ]:
import pandas as pd
# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(link_and_data.items()), columns=['URL', 'Content'])

df['URL'][:5]

0    https://mitmachen.siegburg.de/angebotslandkarte
1                 https://mitmachen.jena.de/projekts
Name: URL, dtype: object

In [ ]:
df.shape

(2, 2)

In [ ]:
# Display the resulting DataFrame to the user
df['Content'][0]

{'https://mitmachen.siegburg.de/proposals/1090-senioiren-cafe-lichtblick-kaldauen': 'Senioiren Cafe Lichtblick Kaldauen18. September 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte60-7475+Geselliges BeisammenseinDas Cafe Lichtblick in Kaldauen lädt alle Senioren zum geselligen Beisammensein bei Kaffee und Kuchen ein. Es wird gesungen, gespielt, getanzt, gelacht und viel erzählt. Auch gemeinsame Ausflüge und Feiern stehen auf dem Programm.Haben Sie Interesse, dann kommen Sie gerne vorbei. Die Senioren und das Team freut sich auf Sie!Treffen : Alle 14 Tage Dienstags von 15.00 - 17.00 Uhr\xa0(immer in geraden Wochen)Ort : Pfarrheim der katholischen Kirche KaldauenAntoniusweg 1 ;\xa053721 SiegburgAnmeldungen bei Rita Quadt ; Tel.:\xa0015733725174:Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.MCafé LichtblickBeigetreten am: 18.09.2024Konto verifiziertVorschläge1Unterstützer*innenUnterstützer*innenUnterstützung0 Unterstützer*innenUm fortzufahren, m

In [ ]:
df['Content'][1]

{'https://mitmachen.jena.de/matschspielplatz-schlippenstrasse': 'Matschspielplatz Schlippenstraße: In Wenigenjena, konkret auf dem \xa0Spielplatz Schlippenstraße, soll der erste Matschspielplatz in Jena entstehen. Der im Vergleich zu anderen Spielplätzen eher kleinere Platz wurde 1990 errichtet und im Jahr 2005 sind zuletzt die Spielgeräte erneuert worden. Nun soll die Gesamtfläche von ca. 600 m² als Matschspielpatz umgestaltet werden. Der Spielplatz soll das ganze Jahr Gruppen- und Einzelspielmöglichkeiten für Kinder von 0 bis 6 Jahren bieten.Bei der Errichtung von Spielplätzen mit dem Element Wasser gilt es nach den Vorschriften, immer auch eng mit den Gesundheitsämtern zusammenzuarbeiten. Das Spielen darf nicht krank machen, sondern soll allen Spaß (und maximal schmutzige Klammotten) bereiten. Deshalb werden wir in Jena keine stehende Gewässer in Form von Wasserbecken oder ähnlichen Elementen anbieten.Planungschritte zur Errichtung des SpielplatzesBürgerbeteiligung Phase 1Im Novembe

In [ ]:
import re

# Enhanced function to extract all logical parts, including "Unterstützer*innen"
def extract_full_data_with_supporters(content):
    # Extract title (everything before the first date)
    title_match = re.search(r'^(.*?)(\r|\d{1,2}\.\s\w+\s\d{4})', content)
    title = title_match.group(1).strip() if title_match else None

    # Extract date
    date_match = re.search(r'\d{1,2}\.\s\w+\s\d{4}', content)
    date = date_match.group(0) if date_match else None

    # Extract comments count
    comments_match = re.search(r'(\d+)\sKommentare', content)
    comments = int(comments_match.group(1)) if comments_match else 0

    # Extract tags (sections with numbers or + signs)
    tags_match = re.findall(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
    tags = ', '.join(tags_match) if tags_match else None

    # Extract description (everything after "Geselliges Beisammensein" or similar patterns)
    description_start = re.search(r'(Geselliges Beisammensein|Angebotslandkarte)', content)
    description = content[description_start.start():].strip() if description_start else None

    # Extract username
    username_match = re.search(r'(\w+\s\w+|Beigetreten am:.*?\d{4})', content)
    username = username_match.group(1).split('Beigetreten am:')[0].strip() if username_match else None

    # Extract Vorschläge count
    vorschlaege_match = re.search(r'Vorschläge(\d+)', content)
    vorschlaege = int(vorschlaege_match.group(1)) if vorschlaege_match else 0

    # Extract Konto verification status
    konto_match = re.search(r'(Konto\s(verifiziert|ist nicht verifiziert))', content)
    konto_status = konto_match.group(2) if konto_match else None

    # # Extract registration date
    # registration_match = re.search(r'Beigetreten am:\s(\d{1,2}\.\s\w+\s\d{4})', content)
    # registration_date = registration_match.group(1) if registration_match else None

    # Extract number of Unterstützer*innen
    supporters_match = re.search(r'(\d+)\sUnterstützer\*in', content)
    supporters = int(supporters_match.group(1)) if supporters_match else 0

    return title, date, comments, tags, description, username, vorschlaege, konto_status, supporters

# Apply the enhanced function to the DataFrame and create new columns
df[['Title', 'Date', 'Comments', 'Tags', 'Description', 'Username', 'Vorschläge', 'Konto Status', 'Supporters']] = df['Content'].apply(
    lambda x: pd.Series(extract_full_data_with_supporters(x))
)


TypeError: expected string or bytes-like object

In [ ]:
df['Comments'].replace(20240, 0, inplace=True)

C:\Users\Rusel\AppData\Local\Temp\ipykernel_15012\2378994937.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Comments'].replace(20240, 0, inplace=True)


In [ ]:
df['Content'][3]

'Geselliges Beisammensein in Braschoß18. August 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte18-2425-4950-5960-7475+Geselliges BeisammenseinDer "Pfarrverein Braschoß" lädt regelmäßig zum geselligen Beisammensein nach Braschoß ein:- Skatabende- Leseabende- Spielabend- ...Konkrete Veranstaltungen werden unterhttps://www.pfarrverein-braschoss.de/aktuelles.htmlveröffentlicht.Gerne informieren wir Sie per Mail über neue Veranstaltungen:https://www.pfarrverein-braschoss.de/newsletter.htmlAlle interessierte Bürger:innen\xa0 sind herzlich willkommen. In der Veranstaltungsübersicht ist vermerkt, wenn vorab eine Anmeldung erforderlich ist.Die Veranstaltungen finden in 53721 Siegburg-Braschoß, Braschosser Str. 50 statt.Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.KEEBeigetreten am: 20.11.2022Konto verifiziertVorschläge3Unterstützer*innenUnterstützer*innenUnterstützung4 Unterstützer*innenUm fortzufahren, müssen Sie sichanmeldenoderregistrieren.Dokumen

In [ ]:
df['URL'][3]

'https://mitmachen.siegburg.de/proposals/1056-geselliges-beisammensein-in-braschoss'

In [ ]:
df['URL'][2]

'https://mitmachen.siegburg.de/proposals/1087-gemeinschaftsgarten-cecilienstrasse'

In [ ]:
# Function to clean description considering keywords, numeric patterns, and refined starting logic
def clean_description_advanced(content):
    # Define keywords that mark the beginning of the description
    keywords = [
        'Geselliges Beisammensein', 'Natur', 'Hilfe & Beratung', 'Bildung', 
        'Musik', 'Bewegung', 'Glaube', 'Kulinarisches', 'Kunst & Kultur', 'Sonstiges',
    ]
    
    # Check for keywords first
    for keyword in keywords:
        if keyword in content:
            start_idx = content.find(keyword) + len(keyword)
            description = content[start_idx:].strip()
            description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
            return description

    # If no keyword is found, check for numeric patterns like "18-24, 25-49, etc."
    numeric_pattern = re.search(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
    if numeric_pattern:
        start_idx = numeric_pattern.end()
        description = content[start_idx:].strip()
        description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
        return description

    # As a fallback, find the first capital letter, quote, or digit to mark the start
    fallback_match = re.search(r'[A-Z"0-9]', content)
    if fallback_match:
        start_idx = fallback_match.start()
        description = content[start_idx:].strip()
        description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
        return description

    # If nothing works, return the content as is
    return content

# Apply the advanced cleaning function to the Description column
df['Description'] = df['Content'].apply(clean_description_advanced)


In [ ]:
df

,URL,Content,Title,Date,Comments,Tags,Description,Username,Vorschläge,Konto Status,Supporters
0,https://mitmachen.siegburg.de/proposals/1090-s...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,0,"60-74, 75+",Das Cafe Lichtblick in Kaldauen lädt alle Seni...,Senioiren Cafe,1,verifiziert,0
1,https://mitmachen.siegburg.de/proposals/1089-d...,Digitalsprechstunde für Senior*innen16. Septem...,Digitalsprechstunde für Senior*innen,16. September 2024,0,"60-74, 75+",Am ersten Donnerstag im Monat erhalten Seniori...,Digitalsprechstunde für,17,verifiziert,0
2,https://mitmachen.siegburg.de/proposals/1087-g...,Gemeinschaftsgarten Cecilienstraße12. Septembe...,Gemeinschaftsgarten Cecilienstraße,12. September 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+, ...",Der Gemeinschaftsgarten ist eine Kooperation d...,Gemeinschaftsgarten Cecilienstraße12,1,ist nicht verifiziert,0
3,https://mitmachen.siegburg.de/proposals/1056-g...,Geselliges Beisammensein in Braschoß18. August...,Geselliges Beisammensein in Braschoß,18. August 2024,0,"18-24, 25-49, 50-59, 60-74, 75+",in Braschoß18. August 20240 KommentareZugehöri...,Geselliges Beisammensein,3,verifiziert,4
4,https://mitmachen.siegburg.de/proposals/1055-b...,Bildungsveranstaltungen Pfarrverein Braschoß18...,Bildungsveranstaltungen Pfarrverein Braschoß,18. August 2024,0,"18-24, 25-49, 50-59, 60-74, 75+",sveranstaltungen Pfarrverein Braschoß18. Augus...,Bildungsveranstaltungen Pfarrverein,3,verifiziert,4
5,https://mitmachen.siegburg.de/proposals/1054-o...,Öffentlicher Bücherschrank für Kinder-/Jugend-...,Öffentlicher Bücherschrank für Kinder-/Jugend-...,15. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+",Stöbern und Tauschen erwünscht!Den Bücherschra...,Öffentlicher Bücherschrank,3,verifiziert,2
6,https://mitmachen.siegburg.de/proposals/1052-r...,Rikscha-Fahrten in Siegburg14. August 20240 Ko...,Rikscha-Fahrten in Siegburg,14. August 2024,0,75+,Ausfahrten mit der Fahrrad Rikscha in die nähe...,Fahrten in,2,ist nicht verifiziert,4
7,https://mitmachen.siegburg.de/proposals/1051-m...,Mach was freiwillig!\r\n\r\nEhrenamt und Engag...,Mach was freiwillig!,12. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+, ...",Die Freiwilligen-Agentur für den Rhein-Sieg-Kr...,Mach was,1,ist nicht verifiziert,2
8,https://mitmachen.siegburg.de/proposals/1049-g...,Generationen-Parcours08. August 20240 Kommenta...,Generationen-Parcours,08. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+",Die Outdoor Fitness Anlage am Michaelsberg unt...,August 20240,17,verifiziert,0
9,https://mitmachen.siegburg.de/proposals/1041-e...,Erlebe Abenteuer mit den Annopfadfinder*innen!...,Erlebe Abenteuer mit den Annopfadfinder*innen!,17. Juli 2024,0,"0-12, 13-17, 18-24",Wir sind diePfadfinder*innen vom Stamm St. Ann...,Erlebe Abenteuer,1,verifiziert,1


In [ ]:
df.to_csv('siegburg_data.csv', index=False)

In [ ]:
df['URL'][14]

'https://mitmachen.siegburg.de/proposals/1030-sommerferien-workshop-self-defense-4-girls'